In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from jupyterplot import ProgressPlot

import torch
from torch import nn
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets as D
from torchvision import transforms as T

from utils import sample_random_data
from utils import show_images
from utils import train_step, test_step
from utils import BaselineModel

# CIFAR10에서 직접 만든 모델 훈련해보기
#### 힌트
- Convolution layer 늘리기: input / output feature의 크기가 같도록 하면 늘릴 수 있음
- Convolution layer의 out_channels 조절해보기
- Fully connected layer(nn.Linear)의 features 조절해보기

In [ ]:
class ConvolutionNeuralNetworks(nn.Module):
    def __init__(self):
        super().__init__()
        #============================#
        #      Your code here        #
        #============================#
        
        #============hint============#
        # conv_layer = nn.Conv2d(
        #     in_channels,
        #     out_channels,
        #     kernel_size,
        #     stride,
        #     padding
        # )
        #
        # maxpool_layer = nn.MaxPool2d(
        #     kernel_size,
        #     stride,
        #     padding
        # )
        #
        # relu = nn.ReLU(inplace=True)
        #
        # fully_connected_lyaer = nn.Linear(
        #     in_features,
        #     out_features
        # )
        #============================#
        
        # 기본 틀
        # 입력은 3 X 32 X 32
        # conv -> relu -> (maxpool / optional) -> fc
        # ex)
        # conv -> relu -> conv -> relu -> mp -> conv -> relu -> fc ...
    
    def forward(self, x):
        # conv / relu / maxpool layers here
        x = x.flatten(start_dim=1)
        # fc layers here
        return x
class ConvolutionNeuralNetworks(BaselineModel):
    pass
cnn = ConvolutionNeuralNetworks()
print(cnn)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_epochs = 30
batch_size = 128
learning_rate = 0.01

# CIFAR10 데이터셋

In [ ]:
phases = ['train', 'test']
transform = T.Compose([T.Resize((32, 32)), T.ToTensor()])
cifar10_dataset = {
    phase: D.CIFAR10(
        'data', train=phase=='train', transform=transform, download=True
    )
    for phase in phases
}
loader = {
    phase: DataLoader(
        cifar10_dataset[phase],
        batch_size=batch_size,
        shuffle=phase=='train'
    )
    for phase in ['train', 'test']
}
images, target = sample_random_data(cifar10_dataset['train'])
titles = [cifar10_dataset['train'].classes[idx] for idx in target]
show_images(images.permute(0,2,3,1), titles)

In [ ]:
cnn = ConvolutionNeuralNetworks().to(device)
optimizer = torch.optim.SGD(cnn.parameters(), learning_rate, 0.9)
criterion = nn.CrossEntropyLoss()

pp = ProgressPlot(
    plot_names=phases,
    line_names=['loss', 'accuracy'],
    x_lim=[0, num_epochs*len(loader['train'])],
    x_label='Iteration',
    y_lim=[[0, 2], [0, 100]]
)

accuracy = 0
for epoch in range(num_epochs):
    for inputs, target in loader['train']:
        loss = train_step(cnn, inputs, target, optimizer, criterion, device)
        pp.update([[loss, -1], [-500, accuracy]])
    
    corrects = 0
    for inputs, target in loader['test']:
        output, _ = test_step(cnn, inputs, target, device=device)
        corrects += (output.argmax(1).cpu() == target).sum().item()
    accuracy = corrects / len(cifar10_dataset['test']) * 100
    
    print(f'Epoch: {epoch+1} accuracy {accuracy:.2f}')

In [ ]:
images, targets = sample_random_data(cifar10_dataset['test'], num=25)
scores = cnn(images.to(device))
predicted = scores.argmax(1).detach().cpu()
correctness = (predicted == torch.as_tensor(targets))

target_names = [cifar10_dataset['test'].classes[t] for t in targets]
pred_names = [cifar10_dataset['test'].classes[t] for t in predicted]
titles = [f'target {t}\npredicted as {p}\n{"correct" if c else "WRONG"}'
          for t, p, c in zip(target_names, pred_names, correctness)]
show_images(images.permute(0,2,3,1), titles)